In [ ]:
import os
import json
import pandas as pd
from addict import Dict
import dateutil
from datetime import datetime, timedelta
from functools import lru_cache
from pdpyras import APISession


In [ ]:
import datetime
from dateutil import parser
#assert 'PD_PAT' in os.environ, 'No API_key'
## if this fails then export
## export PD_PAT=<API KEY>
## or get it from a file and put it in env
with open('pd_models/config.json') as c:
    os.environ['PD_PAT'] = json.load(c)['PD_PAT']

In [ ]:
sess = APISession(api_key=os.environ.get('PD_PAT'))
next = False
all_incidents = []
offset = 0
models = {}

In [ ]:

def make(obj, filter=None):
    offset=0
    more = False
    params = {}
    paged = False
    sess.headers.update({'X-EARLY-ACCESS' : 'UNDER CONSTRUCTION'})
    print('Headers : {} '.format(sess.headers))
    if filter:
        params.update(filter)
    print('params {} '.format(params))


    while True:
            if 'notes' not in obj:
                more = paged = True
                params.update({'total': True, 'offset' : offset })
            _p = sess.get('{}'.format(obj), json=params)
            if _p :
                _j = json.loads(_p.content)
                if 'total' in _j:
                    print('Total: {} , More : {}, Offset {}, Limit {} '.format(_j['total'],
                                                                               _j['more'],
                                                                               _j['offset'],
                                                                               _j['limit']))
                    print('Calling {}, offset: {} '.format(obj, offset))
                    more = _j['more']
                obj_name = obj.split('/')[-1]
                print('Model Object name : {} {}'.format(obj_name, "dddd"))
                for _d in _j[obj_name]:
                    #print('dict : {} '.format(_j))
                    yield ModelBase(obj_name[:-1], (), _d)
                # more = _j['more']
                if not more:
                    break
                offset += 25


In [ ]:
class ModelBase(type):

    def __new__(cls, name, bases, dict):
        for k,v in dict.items():
            setattr(cls, k, v)
            models[name] = cls = super(ModelBase, cls).__new__(cls, name, bases, dict)
            return cls

    def __repr__(self):
         return ('id={}'.format(self.id))

class ModelObjet(metaclass=ModelBase):

    def stop(self):
        pass

class BaseObj(metaclass=ModelBase):

    def __init__(self, **kwargs):
        super(BaseObj, self).__init__(names, (), **kwargs)

    def _get_data(self, uri):
        _p = json.loads(self.session.get('{}'.format(uri)).content)
        _slice = list(_p.keys())[0]
        return _p[_slice]

    def __repr__(self):
        return ('id={}'.format(self.id))


In [ ]:
# build a model for a date range
startdate = dateutil.parser.parse("20/06/2020 12:00:00").isoformat()
enddate = dateutil.parser.parse('25/06/2020 12:00:00').isoformat()
team_filter = 'AO'

In [ ]:
# build incident filter to get all relevant incidents
incident_filter = {
        'since': '{}'.format(startdate),
        'until': '{}'.format(enddate),
        'team_ids': [],
}

print('Incident filter : {}'.format(incident_filter))

In [ ]:
# lookup all teams with the team filter
teams = [t for t in make('/teams') if team_filter or 'AO' in t.name]

print('Got AO teams {}'.format(teams))

In [ ]:
cols = {'team name': [],
            'incident id': [],
            'incident no':[],
            'priority' : [],
            'service': [],
            'created at':[],
            'status':[],
            'alert_all_triggered': [],
            #'resolve_reason': [],
            'title':[]
            }

In [ ]:
for t in teams:
        incident_filter.update({'team_ids' : [t.id]})
        for i in make('/incidents', filter=incident_filter):
            cols['team name'].append(t.name)
            cols['incident id'].append(i.id)
            cols['incident no'].append(i.incident_number)
            if i.priority:
                cols['priority'].append(i.priority['summary'])
            else:
                cols['priority'].append('N/A')
            cols['service'].append(i.service['summary'])
            cols['created at'].append(i.created_at)
            cols['alert_all_triggered'].append(i.alert_counts['all'])
            # if 'resolve_reason' in i and i.resolve_reason:
            #     cols['resolve_reason'].append(i.resolve_reason['type'])
            # else:
            #     cols['resolve_reason'].append('N/A')
            cols['status'].append(i.status)
            cols['title'].append(i.title)

print('Finished building DF')
df = pd.DataFrame(cols)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
ss = df.service
ss.reset_index(drop=True)

In [ ]:
import matplotlib.pyplot as pl
%matplotlib inline

In [ ]:
srv = df['service'].value_counts()
srv.plot()